<h1>SIOS Observation Facility Catalogue</h1>

<h3>This is a short notebook to query the SIOS Observation Facility catalogue API.</h3>

The catalogue is espose on the SIOS website (https://sios-svalbard.org/sios-ri-catalogue) in a human readable form, and it is also connected to a REST API which returns a json file containing the information about the Observation Facilities.
This catalogue has been developed to provide an overview of the observation facilities which collect SIOS data. An observation facility can be one instrument or a collection of instruments, e.g. a weather station, and is a term used by the World Meteorological Organization (WMO). The annotation is standardised following the WMO standards as far as possible, in order to make entries unambiguous and interoperable internationally.

This notebook is connecting to the API and displaying the records on a map, including some basic information. More information can be found clicking on the URLs in the popup, or accessing directly the Observation Facility Catalogue.

In [1]:
import requests
import pandas as pd
import folium
from folium.plugins import Search
import shapely.wkt

def sios_observation_facilities():

    response = requests.get('https://sios-svalbard.org/sios-ri-catalogue/rest/sios-ri-catalogue.json')  
    ofs = []
    ofs_list = []
    for of in response.json():
        if of['title'] not in ofs_list:
            ofs.append({'title':of['title'],
                        'station type':of['OF-Type'], 
                        'geometry': of['OF-Coordinates'], 
                        'station height':of['OF-Height'],
                        'sios-url':of['OF-SIOS-URL'],
                        'institution':of['institution'],
                        'variables':[of['Observed-Variable'].replace('&gt;', ' > ')]})
            ofs_list.append(of['title'])
        else:
            st = next(item for item in ofs if item["title"] == of['title'])
            st['variables'].append(of['Observed-Variable'].replace('&gt;', ' > '))
    return ofs

observation_facilities = sios_observation_facilities()

m = folium.Map(location=[78, 15], zoom_start=6, tiles='CartoDB positron')

df = pd.DataFrame(observation_facilities)

for i, r in df.iterrows():
    html = """
    <!DOCTYPE html>
    <html>
    <table style="width:300px;">
        <tr><td colspan="2"><a target='blank' href={url}>{title}</a></td></tr>
        <tr><td>instituion:</td><td>{institution}</td></tr>
        <tr><td>station type:</td><td>{stype}</td></tr>
        <tr style="border-bottom:1px solid black"><td>geometry:</td><td>{geometry}</td></tr>
        <tr><td>variables ({vlen}):</td><td>{variables}</td></tr>
    </table>
    </html> 
    """.format(url=df['sios-url'][i], 
               title=df['title'][i],
               geometry=df['geometry'][i],
               stype=df['station type'][i],
               institution=df['institution'][i],
               variables=df['variables'][i],
               vlen = len(df['variables'][i]))
    
    el = folium.GeoJson(data=shapely.wkt.loads(r["geometry"]), 
                        tooltip=r['title'])
    el.add_child(folium.Popup(html))
    el.add_to(m)

    
display(m)

In [2]:
df

,title,station type,geometry,station height,sios-url,institution,variables
0,COSMOS - black carbon,land platform (fixed),POINT (11.88611 78.90721),475.00,https://sios-svalbard.org/node/1040,National Institute for Polar Research,[ATMOSPHERE > AEROSOLS > CARBONACEOUS AEROSOLS]
1,Mast for Temperature Gradient,land platform (fixed),POINT (11.83393447 78.92080104),10.00,https://sios-svalbard.org/node/927,Alfred Wegener Institute,[ATMOSPHERE > ATMOSPHERIC TEMPERATURE > SURFAC...
2,Medusa,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60359,Norwegian Institute for Air Research,[ATMOSPHERE > ATMOSPHERIC CHEMISTRY > HALOCARB...
3,Nephlometer,land platform (fixed),POINT (11.88611 78.90721),475.00,https://sios-svalbard.org/node/1007,Stockholm University,[ATMOSPHERE > AEROSOLS > AEROSOL RADIANCE]
4,10-line Global Sampler,land platform (fixed),POINT (11.89418 78.9176),45.00,https://sios-svalbard.org/node/1025,National Institute for Polar Research,"[ATMOSPHERE > AEROSOLS > PARTICULATE MATTER, A..."
...,...,...,...,...,...,...,...
226,Zeppelin Aerodynamic Particle Sizer,land platform (fixed),POINT (11.88934 78.90669),475.00,https://sios-svalbard.org/node/1065,Norwegian Institute for Air Research,[ATMOSPHERE > AEROSOLS > AEROSOL PARTICLE PROP...
227,Zeppelin Europakasse sampler,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60355,Norwegian Institute for Air Research,[ATMOSPHERE > ATMOSPHERIC CHEMISTRY > SULFUR C...
228,Zeppelin Ozone Analyser,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60354,Norwegian Institute for Air Research,[ATMOSPHERE > AIR QUALITY > TROPOSPHERIC OZONE]
229,Zeppelin Picarro,land platform (fixed),POINT (11.88579 78.90724),473.00,https://sios-svalbard.org/node/60360,Norwegian Institute for Air Research,[ATMOSPHERE > ATMOSPHERIC CHEMISTRY > CARBON A...
